### calcul multiprocessing

In [ ]:
import os, sys
from re import sub
import pandas as pd
import numpy as np
from latloncalc.latlon import LatLon, Latitude, Longitude
from itertools import combinations
from multiprocessing import Pool, cpu_count, current_process